# Spark DataFrames

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('Spark DataFrames') \
    .getOrCreate()
sc = spark.sparkContext

## DataFrames

A distributed collection of data grouped into named columns.  
A DataFrame is equivalent to a relational table in Spark SQL.

### Differences vs RDDs

Contrary to Spark's RDDs, DataFrames are not schema-less.

#### Pros of DataFrames vs RDDs

- they enforce a schema
- you can run SQL queries against them
- faster than RDDs
- much smaller than RDDs when stored in parquet format

### Creation

#### From a RDD

In [2]:
numbers = [i for i in range(10)]
numbers_rdd = sc.parallelize(numbers)

In [3]:
tuples_rdd = sc.parallelize([
    ('banana', 4), ('orange', 12), ('apple', 3),
    ('pineapple', 1), ('banana', 3), ('orange', 6)])
tuples_rdd

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:262

In [4]:
tuples_rdd.collect()

[('banana', 4),
 ('orange', 12),
 ('apple', 3),
 ('pineapple', 1),
 ('banana', 3),
 ('orange', 6)]

In [5]:
df_fruits = spark.createDataFrame(tuples_rdd) 
df_fruits.head(3)

[Row(_1='banana', _2=4), Row(_1='orange', _2=12), Row(_1='apple', _2=3)]

In [6]:
df_fruits.show()

+---------+---+
|       _1| _2|
+---------+---+
|   banana|  4|
|   orange| 12|
|    apple|  3|
|pineapple|  1|
|   banana|  3|
|   orange|  6|
+---------+---+



#### From a pandas DataFrame

In [7]:
import pandas as pd
data_dict = {'a': 1, 'b': 2, 'c': 3}
pandas_df = pd.DataFrame.from_dict(
    data_dict, orient='index', columns=['position'])
pandas_df

,position
a,1
b,2
c,3


In [8]:
spark_df = spark.createDataFrame(pandas_df)
spark_df

DataFrame[position: bigint]

In [9]:
spark_df.show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



### Running sql queries against DataFrames

In [10]:
spark_df.createOrReplaceTempView('my_table')

In [11]:
spark.sql("SELECT * FROM my_table")

DataFrame[position: bigint]

This will return a `DataFrame`, just like a RDD, **is not computed until an action is called**.

### Actions
All actions perform computations, some like `show` or `printSchema` print out to stdout, some, like `count` will return a value.

#### `.show(...)`
Prints out the first 20 values of the DataFrame.

In [12]:
spark_df.show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



Default can be changed.

In [13]:
spark_df.show(2)

+--------+
|position|
+--------+
|       1|
|       2|
+--------+
only showing top 2 rows



#### `.printSchema()`
Prints out the schema of the DataFrame.

In [14]:
spark_df.printSchema()

root
 |-- position: long (nullable = true)



In [15]:
# TODO: explain the concept of schema, and say something about columns

In [16]:
spark_df.columns  # not an `action` (nor a transformation)

['position']

#### `.take(...)`
Compute the first n values of the DataFrame.

In [17]:
spark_df.take(5)

[Row(position=1), Row(position=2), Row(position=3)]

As you can see, a PySpark `DataFrame` is a collection of [`Row`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.Row) objects (cf [doc](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.Row)).

#### `.collect(...)`
Like `.take(...)` but will take effect on all rows of the DataFrame.

In [18]:
spark_df.collect()

[Row(position=1), Row(position=2), Row(position=3)]

---
⚠️ `.collect()` will collect all the values, do **NOT** perform this action on a full DataFrame, only small DataFrames like aggregate results.

---

#### `.count(...)`
Return the number of `Rows` in the DataFrame

In [19]:
spark_df.count()

3

### Transformations
- [PySpark Cookbook](https://subscription.packtpub.com/book/big_data_and_business_intelligence/9781788835367/3/ch03lvl1sec34/overview-of-dataframe-transformations)

#### `.take(...)`

In [20]:
spark_df.take(3)

[Row(position=1), Row(position=2), Row(position=3)]

In [21]:
# Equivalent ?
spark_df.head(5)

[Row(position=1), Row(position=2), Row(position=3)]

#### `.collect(...)`

In [22]:
spark_df.collect()

[Row(position=1), Row(position=2), Row(position=3)]

### Missing values: `.na`

In [23]:
spark_df.na

#### `.fill(...)`

In [24]:
spark_df.na.fill(0).show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



#### `.drop()`

In [25]:
spark_df.na.drop().show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



Equivalent to `.dropna()`

In [26]:
spark_df.dropna().show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



Optional parameter, select a `subset` of columns.

#### `.replace(...)`

In [27]:
spark_df.na.replace(2, 4).show() 

+--------+
|position|
+--------+
|       1|
|       4|
|       3|
+--------+



#### `.select()`

In [28]:
spark_df.printSchema()

root
 |-- position: long (nullable = true)



In [29]:
spark_df.select('position').show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



Similar to `spark.sql.select("SELECT position FROM my_table")`.  

To claim equivalence, we would have to check the execution plan of both.

In [30]:
spark_df.select('position').explain()

== Physical Plan ==
*(1) Scan ExistingRDD[position#13L]




In [31]:
spark.sql("SELECT * FROM my_table").explain()

== Physical Plan ==
*(1) Scan ExistingRDD[position#13L]




### Some differences with pandas' DataFrames

- transformations in PySpark are not `inplace` (immutable)

Accessors are returning `Column`

In [32]:
spark_df.position, spark_df['position']

(Column<b'position'>, Column<b'position'>)

Not very useful by themselves, but can be passed to a `.select(...)`.

In [33]:
spark_df.select(spark_df.position).show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



But in a case like this, just like SQL, the executor can infer the "table", this will work:

In [34]:
spark_df.select('position')

DataFrame[position: bigint]

In [35]:
spark_df.select('position').show()

+--------+
|position|
+--------+
|       1|
|       2|
|       3|
+--------+



#### `.alias(...)`

In [36]:
spark_df.select(spark_df.position.alias('aliased_column')).show()

+--------------+
|aliased_column|
+--------------+
|             1|
|             2|
|             3|
+--------------+



#### `.drop(...)`

In [37]:
spark_df.drop('position')

DataFrame[]

In [40]:
#spark_df = spark_df.drop('position')

In [41]:
#spark_df.select('position').show()